<a href="https://colab.research.google.com/github/stephyi/Rossman-Pharmaceuticals-Sales-Prediction/blob/master/rossman_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# to display all columns
pd.set_option('display.max.columns', None)

# to display the entire contents of a cell
pd.set_option('display.max_colwidth', None)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [ ]:
#changing the working directory
%cd /content/gdrive/My Drive/Kaggle
!pwd


/content/gdrive/My Drive/Kaggle
/content/gdrive/My Drive/Kaggle


In [ ]:
# load and preview dataset
store = pd.read_csv('store.csv')
store.sample(3)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
222,223,d,c,2920.0,10.0,1995.0,1,27.0,2011.0,"Jan,Apr,Jul,Oct"
493,494,b,a,1260.0,6.0,2011.0,0,NaN,NaN,NaN
242,243,a,a,310.0,NaN,NaN,1,5.0,2013.0,"Feb,May,Aug,Nov"


In [ ]:
# load and preview dataset
train = pd.read_csv('train.csv')
train.sample(3)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
615038,344,5,2013-12-27,7832,1182,1,0,0,1
814335,56,1,2013-07-01,8455,526,1,1,0,0
758249,835,3,2013-08-21,3425,516,1,0,0,1


In [ ]:
# load and preview dataset
test = pd.read_csv('test.csv')
test.sample(3)

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
8227,8228,677,2,2015-09-08,1.0,0,0,1
34305,34306,84,6,2015-08-08,1.0,0,0,0
30055,30056,128,4,2015-08-13,1.0,0,0,1


In [ ]:
def preprocess(data):
  data['Date'] = pd.to_datetime(data['Date'],format = '%Y-%m-%d')

  data['day']=data['Date'].dt.day
  data['month']=data['Date'].dt.month
  data['year']=data['Date'].dt.year

  df_train_store=pd.merge(store,data,how='left',on='Store')


  map_promo= {'Jan,Apr,Jul,Oct':1,'Feb,May,Aug,Nov':2,'Mar,Jun,Sept,Dec':3}
  df_train_store['PromoInterval']=df_train_store['PromoInterval'].map(map_promo)

  df_train_store['StateHoliday'].replace({0:'o','0':'o'},inplace=True)
  df_train_store['StateHoliday'].replace({'0':'o'},inplace=True)

  df_train_store = df_train_store[df_train_store["Open"] != 0]

  df_train_store=df_train_store.drop(['Date','Open','Store'],axis=1)

  return df_train_store

In [ ]:
df_train_store=preprocess(train)

In [ ]:
X=df_train_store.drop(['Sales'],axis=1)
y=np.log1p(df_train_store.Sales)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:

##SKLEARN MODEL PIPELINE


from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

##Function to preprocess the data

num_pipeline = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                    #('drop_attributes', AttributeDeleter()),
                                    #('std_scaler', StandardScaler())
                            ])
categorical_pipeline=Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                                        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

categorical_features = df_train_store.select_dtypes(include=['object']).columns
numerical_features = df_train_store.select_dtypes(include=['float64', 'int64']).drop(['Sales'],axis=1).columns

full_pipeline = ColumnTransformer(
        transformers=[('num', num_pipeline, numerical_features),
                     ('cat', categorical_pipeline, categorical_features)])

    

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = Pipeline(steps=[('preprocessor', full_pipeline),
                      ('regressor', RandomForestRegressor())])

In [ ]:

rf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                             

In [ ]:
y_pred = rf.predict(X_test)


In [ ]:
## Score of our model

from sklearn.metrics import r2_score, mean_squared_error


r2_score(y_test,y_pred)

0.9778283945794213

In [ ]:
## Calculating the MSE
mean_squared_error(y_test,y_pred)

0.004153937312487527

In [ ]:
## Calculating the loss
np.sqrt(mean_squared_error(y_test,y_pred))

0.06445104586030802

In [ ]:
!pip install pickle-mixin

In [ ]:
import pickle
# Saving model to disk
pickle.dump(rf, open('model.pkl','wb'))

# Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))
